## TITLE

In [1]:
from LLMs_attack import *
import pickle

print(torch.__version__)  # Check PyTorch version
print(torch.cuda.is_available())  # Doit afficher True
print(torch.version.cuda)  # Doit afficher une version de CUDA
print(torch.cuda.get_device_name(0))  # Doit afficher le nom de votre GPU

2.6.0+cu118
True
11.8
NVIDIA GeForce RTX 4060 Laptop GPU


In [5]:
# WARNING : WILL OVERRRIDE THE PREVIOUS FILE
def save():
    with open("accuracy.pkl", "wb") as f:
        pickle.dump(accuracy, f)
        print("accuracy.pkl saved")

In [6]:
def load():
    if accuracy == {}:
        try:
            with open("accuracy.pkl", "rb") as f:
                accuracy = pickle.load(f)
                print("Loaded accuracy.pkl")
        except:
            print("Could not load accuracy.pkl")

    print(accuracy)
    print(accuracy['standard'][0]['qwen1'][0])

In [7]:
args = argparse.Namespace(
ntrain=0,
data_dir="data/MMLU",
engine=["qwen1"],
n_reduced=None,
use_subset=True,
subset_size = 2,
permutation_attack=False,
position_permute=False,
reduce_attack=False,
load_in_8bit=False,
verbose = True,
stop_attack = False,
subset_subjects = 2,
)

In [8]:
main(args)

['abstract_algebra', 'anatomy']
Namespace(ntrain=0, data_dir='data/MMLU', engine=['qwen1'], n_reduced=None, use_subset=True, subset_size=2, permutation_attack=False, position_permute=False, reduce_attack=False, load_in_8bit=False, verbose=True, stop_attack=False, subset_subjects=2)
Engine: qwen1


Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.



Prompt:  The following are multiple choice questions (with answers) about abstract_algebra.

Find the degree for the given field extension Q(sqrt(2), sqrt(3), sqrt(18)) over Q.
A. 0
B. 4
C. 2
D. 6
Answer:

Predicted:  B
True:  B
Probabilities:  [1.1324882507324219e-06, 3.4749507904052734e-05, 2.682209014892578e-06, 1.9669532775878906e-06]


Prompt:  The following are multiple choice questions (with answers) about abstract_algebra.

Let p = (1, 2, 5, 4)(2, 3) in S_5 . Find the index of <p> in S_5.
A. 8
B. 2
C. 24
D. 120
Answer:

Predicted:  B
True:  C
Probabilities:  [9.894371032714844e-06, 2.0444393157958984e-05, 1.9669532775878906e-06, 8.64267349243164e-06]

Average accuracy 50.00 - abstract_algebra

Prompt:  The following are multiple choice questions (with answers) about anatomy.

A lesion causing compression of the facial nerve at the stylomastoid foramen will cause ipsilateral
A. paralysis of the facial muscles.
B. paralysis of the facial muscles and loss of taste.
C. paralysis o

({'qwen1': [np.float64(0.5), np.float64(0.0)]},
 {'qwen1': [array([ True, False]), array([False, False])]},
 {})

In [9]:
args.permutation_attack = True
args.subset_size = 1
args.subset_subjects = 1

In [10]:
main(args)

['abstract_algebra']
Namespace(ntrain=0, data_dir='data/MMLU', engine=['qwen1'], n_reduced=None, use_subset=True, subset_size=1, permutation_attack=True, position_permute=False, reduce_attack=False, load_in_8bit=False, verbose=True, stop_attack=False, subset_subjects=1)
Engine: qwen1
#####################################################################
Question:  Find the degree for the given field extension Q(sqrt(2), sqrt(3), sqrt(18)) over Q.
Permutation:  0

Prompt:  The following are multiple choice questions (with answers) about abstract_algebra.

Find the degree for the given field extension Q(sqrt(2), sqrt(3), sqrt(18)) over Q.
A. 0
B. 4
C. 2
D. 6
Answer:

Predicted:  B
True:  B
Probabilities:  [1.1324882507324219e-06, 3.4749507904052734e-05, 2.682209014892578e-06, 1.9669532775878906e-06]

Permutation:  1

Prompt:  The following are multiple choice questions (with answers) about abstract_algebra.

Find the degree for the given field extension Q(sqrt(2), sqrt(3), sqrt(18)) over 

({'qwen1': []}, {'qwen1': [array([0])]}, {'qwen1': [array([0.375])]})

In [11]:
args.subset_size = 100
args.subset_subjects = 10
args.verbose = False
args.permutation_attack = False
args.engine = ["qwen1","llama-3b"]
accuracy = {}

In [12]:
accuracy['standard'] = main(args)

['abstract_algebra', 'anatomy', 'astronomy', 'business_ethics', 'clinical_knowledge', 'college_biology', 'college_chemistry', 'college_computer_science', 'college_mathematics', 'college_medicine']
Namespace(ntrain=0, data_dir='data/MMLU', engine=['qwen1', 'llama-3b'], n_reduced=None, use_subset=True, subset_size=100, permutation_attack=False, position_permute=False, reduce_attack=False, load_in_8bit=False, verbose=False, stop_attack=False, subset_subjects=10)
Engine: qwen1
Average accuracy 36.00 - abstract_algebra
Average accuracy 24.00 - anatomy
Average accuracy 39.00 - astronomy
Average accuracy 41.00 - business_ethics
Average accuracy 40.00 - clinical_knowledge
Average accuracy 28.00 - college_biology
Average accuracy 42.00 - college_chemistry
Average accuracy 38.00 - college_computer_science
Average accuracy 42.00 - college_mathematics
Average accuracy 33.00 - college_medicine
Average accuracy: 36.30



Engine: llama-3b


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Average accuracy 35.00 - abstract_algebra
Average accuracy 62.00 - anatomy
Average accuracy 67.00 - astronomy
Average accuracy 55.00 - business_ethics
Average accuracy 67.00 - clinical_knowledge
Average accuracy 74.00 - college_biology
Average accuracy 42.00 - college_chemistry
Average accuracy 47.00 - college_computer_science
Average accuracy 34.00 - college_mathematics
Average accuracy 61.00 - college_medicine
Average accuracy: 54.40





In [13]:
save()

accuracy.pkl saved


In [14]:
args.subset_size = 50
args.position_permute = True
accuracy['pos_permute'] = main(args)

['abstract_algebra', 'anatomy', 'astronomy', 'business_ethics', 'clinical_knowledge', 'college_biology', 'college_chemistry', 'college_computer_science', 'college_mathematics', 'college_medicine']
Namespace(ntrain=0, data_dir='data/MMLU', engine=['qwen1', 'llama-3b'], n_reduced=None, use_subset=True, subset_size=50, permutation_attack=False, position_permute=True, reduce_attack=False, load_in_8bit=False, verbose=False, stop_attack=False, subset_subjects=10)
Engine: qwen1
Average accuracy 12.00 - abstract_algebra
Average accuracy 58.00 - abstract_algebra
Average accuracy 18.00 - abstract_algebra
Average accuracy 62.00 - abstract_algebra
Average accuracy 2.00 - anatomy
Average accuracy 44.00 - anatomy
Average accuracy 0.00 - anatomy
Average accuracy 64.00 - anatomy
Average accuracy 10.00 - astronomy
Average accuracy 50.00 - astronomy
Average accuracy 2.00 - astronomy
Average accuracy 74.00 - astronomy
Average accuracy 32.00 - business_ethics
Average accuracy 52.00 - business_ethics
Avera

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Average accuracy 28.00 - abstract_algebra
Average accuracy 46.00 - abstract_algebra
Average accuracy 40.00 - abstract_algebra
Average accuracy 66.00 - abstract_algebra
Average accuracy 56.00 - anatomy
Average accuracy 60.00 - anatomy
Average accuracy 50.00 - anatomy
Average accuracy 64.00 - anatomy
Average accuracy 66.00 - astronomy
Average accuracy 74.00 - astronomy
Average accuracy 68.00 - astronomy
Average accuracy 80.00 - astronomy
Average accuracy 50.00 - business_ethics
Average accuracy 70.00 - business_ethics
Average accuracy 70.00 - business_ethics
Average accuracy 78.00 - business_ethics
Average accuracy 62.00 - clinical_knowledge
Average accuracy 72.00 - clinical_knowledge
Average accuracy 64.00 - clinical_knowledge
Average accuracy 70.00 - clinical_knowledge
Average accuracy 70.00 - college_biology
Average accuracy 80.00 - college_biology
Average accuracy 82.00 - college_biology
Average accuracy 82.00 - college_biology
Average accuracy 28.00 - college_chemistry
Average accur

In [15]:
save()

accuracy.pkl saved


In [16]:
args.position_permute = False
args.permutation_attack = True
accuracy['attack'] = main(args)

['abstract_algebra', 'anatomy', 'astronomy', 'business_ethics', 'clinical_knowledge', 'college_biology', 'college_chemistry', 'college_computer_science', 'college_mathematics', 'college_medicine']
Namespace(ntrain=0, data_dir='data/MMLU', engine=['qwen1', 'llama-3b'], n_reduced=None, use_subset=True, subset_size=50, permutation_attack=True, position_permute=False, reduce_attack=False, load_in_8bit=False, verbose=False, stop_attack=False, subset_subjects=10)
Engine: qwen1
Average accuracy 0.00 - abstract_algebra
Average accuracy 0.00 - anatomy
Average accuracy 0.00 - astronomy
Average accuracy 2.00 - business_ethics
Average accuracy 2.00 - clinical_knowledge
Average accuracy 0.00 - college_biology
Average accuracy 0.00 - college_chemistry
Average accuracy 2.00 - college_computer_science
Average accuracy 2.00 - college_mathematics
Average accuracy 2.00 - college_medicine
Average accuracy: 1.00



Engine: llama-3b


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Average accuracy 6.00 - abstract_algebra
Average accuracy 42.00 - anatomy
Average accuracy 48.00 - astronomy
Average accuracy 34.00 - business_ethics
Average accuracy 40.00 - clinical_knowledge
Average accuracy 50.00 - college_biology
Average accuracy 16.00 - college_chemistry
Average accuracy 14.00 - college_computer_science
Average accuracy 6.00 - college_mathematics
Average accuracy 30.00 - college_medicine
Average accuracy: 28.60





In [17]:
save()

accuracy.pkl saved
